In [6]:
import pandas as pd
import seaborn as sns
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [7]:
data = pd.read_csv("parlay_data_sample.csv")

In [8]:
data.head()

,loan_id,period_start,probability_of_default,loss_given_default,expected_loss,lumos_score,rate_type,franchise,franchise_name,borrstate,...,EXPLANATION_8_ACTUAL_VALUE,EXPLANATION_8_QUALITATIVE_STRENGTH,EXPLANATION_9_FEATURE_NAME,EXPLANATION_9_STRENGTH,EXPLANATION_9_ACTUAL_VALUE,EXPLANATION_9_QUALITATIVE_STRENGTH,EXPLANATION_10_FEATURE_NAME,EXPLANATION_10_STRENGTH,EXPLANATION_10_ACTUAL_VALUE,EXPLANATION_10_QUALITATIVE_STRENGTH
0,0,7/31/20,1.38%,18.39%,0.25%,100,F,False,NaN,KS,...,'7.1',++,state_labor_force,0.079548,'1489001.0',++,t10yie,-0.078435,'1.55',--
1,1,11/30/19,1.83%,30.18%,0.55%,94,V,False,NaN,IL,...,'IL',+,borr_count,-0.047088,'2.0',-,t10yie,-0.045048,'1.61',-
2,2,8/31/15,5.05%,66.34%,3.35%,81,V,False,NaN,CO,...,'4a',++,county_employment,-0.074971,'29960.0',--,t10yie,-0.066798,'1.63',--
3,3,12/31/09,3.92%,42.93%,1.68%,86,V,False,NaN,CT,...,'V',++,vix_change,0.063983,'-45.8',++,loan_max_fico,-0.056703,'732.0',--
4,4,10/31/18,2.96%,47.49%,1.41%,88,F,False,NaN,CA,...,'F',--,county_employment,0.102569,'1573536.0',++,empoyessexistingqty,0.097354,'1.0',++


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 90 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   loan_id                               100000 non-null  int64  
 1   period_start                          100000 non-null  object 
 2   probability_of_default                100000 non-null  object 
 3   loss_given_default                    100000 non-null  object 
 4   expected_loss                         100000 non-null  object 
 5   lumos_score                           100000 non-null  int64  
 6   rate_type                             100000 non-null  object 
 7   franchise                             100000 non-null  bool   
 8   franchise_name                        4410 non-null    object 
 9   borrstate                             100000 non-null  object 
 10  businesstype                          100000 non-null  object 
 11  b

In [12]:
df1 = data.copy()

In [13]:
df1.head(5)

,loan_id,period_start,probability_of_default,loss_given_default,expected_loss,lumos_score,rate_type,franchise,franchise_name,borrstate,...,EXPLANATION_8_ACTUAL_VALUE,EXPLANATION_8_QUALITATIVE_STRENGTH,EXPLANATION_9_FEATURE_NAME,EXPLANATION_9_STRENGTH,EXPLANATION_9_ACTUAL_VALUE,EXPLANATION_9_QUALITATIVE_STRENGTH,EXPLANATION_10_FEATURE_NAME,EXPLANATION_10_STRENGTH,EXPLANATION_10_ACTUAL_VALUE,EXPLANATION_10_QUALITATIVE_STRENGTH
0,0,7/31/20,1.38%,18.39%,0.25%,100,F,False,NaN,KS,...,'7.1',++,state_labor_force,0.079548,'1489001.0',++,t10yie,-0.078435,'1.55',--
1,1,11/30/19,1.83%,30.18%,0.55%,94,V,False,NaN,IL,...,'IL',+,borr_count,-0.047088,'2.0',-,t10yie,-0.045048,'1.61',-
2,2,8/31/15,5.05%,66.34%,3.35%,81,V,False,NaN,CO,...,'4a',++,county_employment,-0.074971,'29960.0',--,t10yie,-0.066798,'1.63',--
3,3,12/31/09,3.92%,42.93%,1.68%,86,V,False,NaN,CT,...,'V',++,vix_change,0.063983,'-45.8',++,loan_max_fico,-0.056703,'732.0',--
4,4,10/31/18,2.96%,47.49%,1.41%,88,F,False,NaN,CA,...,'F',--,county_employment,0.102569,'1573536.0',++,empoyessexistingqty,0.097354,'1.0',++


# Handling Explanation Terms

In [14]:
# Mapping from symbols to descriptive text
qualitative_strength_mapping = {
    '-': 'decrease a little',
    '--': 'decrease moderately',
    '---': 'decrease substantially',
    '+': 'increase a little',
    '++': 'increase moderately',
    '+++': 'increase substantially'
}

# New DataFrame to store the reformatted explanations
df_exp = pd.DataFrame()

# Loop through the explanation indices (1 through 10, assuming they all exist and follow this format)
for i in range(1, 11):  # Adjust the range according to the actual number of explanations
    feature_col = f'EXPLANATION_{i}_FEATURE_NAME'
    strength_col = f'EXPLANATION_{i}_QUALITATIVE_STRENGTH'
    
    # Check if the columns exist in df1, to handle cases where some indices may not be present
    if feature_col in df1.columns and strength_col in df1.columns:
        # Map the qualitative strength descriptions
        df1[strength_col] = df1[strength_col].map(qualitative_strength_mapping)
        
        # Create new column in df_exp combining feature name and mapped strength description
        df_exp[f'EXPLANATION_{i}'] = df1[feature_col] + ': ' + df1[strength_col]
        
print(df_exp)

                                EXPLANATION_1  \
0          business_type: decrease moderately   
1      loan_mean_fico: decrease substantially   
2       loan_max_fico: increase substantially   
3                 t10y2y: increase moderately   
4       loan_max_fico: increase substantially   
...                                       ...   
99995     loan_mean_fico: decrease moderately   
99996      loan_max_fico: increase moderately   
99997   loan_max_fico: increase substantially   
99998          t10y2y: decrease substantially   
99999      borr_count: decrease substantially   

                                           EXPLANATION_2  \
0                         borrstate: increase moderately   
1                  business_type: increase substantially   
2                    loan_mean_fico: increase moderately   
3                     loan_min_fico: decrease moderately   
4      state_civilian_labor_population: decrease mode...   
...                                                

In [15]:
df_exp.head(5)

,EXPLANATION_1,EXPLANATION_2,EXPLANATION_3,EXPLANATION_4,EXPLANATION_5,EXPLANATION_6,EXPLANATION_7,EXPLANATION_8,EXPLANATION_9,EXPLANATION_10
0,business_type: decrease moderately,borrstate: increase moderately,t10y2y: decrease moderately,loan_min_fico: increase moderately,loan_max_fico: decrease moderately,2017_naics_categorical: increase moderately,naics_1: increase moderately,state_unemployment_rate: increase moderately,state_labor_force: increase moderately,t10yie: decrease moderately
1,loan_mean_fico: decrease substantially,business_type: increase substantially,loan_max_fico: decrease moderately,rate_type: increase moderately,naics_3: decrease moderately,t10y2y: decrease moderately,naics_2: increase a little,borrstate: increase a little,borr_count: decrease a little,t10yie: decrease a little
2,loan_max_fico: increase substantially,loan_mean_fico: increase moderately,business_type: increase moderately,rate_type: increase moderately,loan_min_fico: increase moderately,county_unemployment: decrease moderately,fed_funds: decrease moderately,naics_1: increase moderately,county_employment: decrease moderately,t10yie: decrease moderately
3,t10y2y: increase moderately,loan_min_fico: decrease moderately,bankruptcy: decrease moderately,loan_mean_fico: decrease moderately,county_unemployment_rate: increase moderately,empoyessexistingqty: increase moderately,state_unemployment_rate: increase moderately,rate_type: increase moderately,vix_change: increase moderately,loan_max_fico: decrease moderately
4,loan_max_fico: increase substantially,state_civilian_labor_population: decrease mode...,loan_min_fico: increase moderately,loan_mean_fico: increase moderately,bankruptcy: decrease moderately,county_labor_force: increase moderately,business_type: increase moderately,rate_type: decrease moderately,county_employment: increase moderately,empoyessexistingqty: increase moderately


# Handling Other Attributes

In [16]:
df = data[data.columns[:50]]

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 50 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   loan_id                               100000 non-null  int64  
 1   period_start                          100000 non-null  object 
 2   probability_of_default                100000 non-null  object 
 3   loss_given_default                    100000 non-null  object 
 4   expected_loss                         100000 non-null  object 
 5   lumos_score                           100000 non-null  int64  
 6   rate_type                             100000 non-null  object 
 7   franchise                             100000 non-null  bool   
 8   franchise_name                        4410 non-null    object 
 9   borrstate                             100000 non-null  object 
 10  businesstype                          100000 non-null  object 
 11  b

In [18]:
df[['probability_of_default','loss_given_default','expected_loss','collateral_amount','loan_size']]

,probability_of_default,loss_given_default,expected_loss,collateral_amount,loan_size
0,1.38%,18.39%,0.25%,0-200k,101k-500k
1,1.83%,30.18%,0.55%,201k-1M,0-100k
2,5.05%,66.34%,3.35%,201k-1M,0-100k
3,3.92%,42.93%,1.68%,201k-1M,501k+
4,2.96%,47.49%,1.41%,201k-1M,501k+
...,...,...,...,...,...
99995,0.57%,74.76%,0.43%,NaN,501k+
99996,15.09%,92.74%,13.99%,NaN,0-100k
99997,4.51%,63.45%,2.86%,NaN,101k-500k
99998,2.36%,64.90%,1.53%,NaN,101k-500k


**Data Preprocessing**

Converted percentage strings to floats for probability_of_default, expected_loss, and loss_given_default columns to make them usable for analysis.
Converted period_start to datetime format for time-series analysis.
Filtered the dataset to the first 50 columns for focused analysis, likely due to computational efficiency or relevance to the analysis goals.

In [19]:
df['probability_of_default'] = df['probability_of_default'].apply(lambda x: float(x.strip("%")))
df['expected_loss'] = df['expected_loss'].apply(lambda x: float(x.strip("%")))
df['loss_given_default'] = df['loss_given_default'].apply(lambda x: float(x.strip("%")))

In [20]:
df[['collateral_amount','loan_size']]

,collateral_amount,loan_size
0,0-200k,101k-500k
1,201k-1M,0-100k
2,201k-1M,0-100k
3,201k-1M,501k+
4,201k-1M,501k+
...,...,...
99995,NaN,501k+
99996,NaN,0-100k
99997,NaN,101k-500k
99998,NaN,101k-500k


In [21]:
df['period_start'] = pd.to_datetime(df['period_start'])

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 50 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   loan_id                               100000 non-null  int64         
 1   period_start                          100000 non-null  datetime64[ns]
 2   probability_of_default                100000 non-null  float64       
 3   loss_given_default                    100000 non-null  float64       
 4   expected_loss                         100000 non-null  float64       
 5   lumos_score                           100000 non-null  int64         
 6   rate_type                             100000 non-null  object        
 7   franchise                             100000 non-null  bool          
 8   franchise_name                        4410 non-null    object        
 9   borrstate                             100000 non-null  objec

<li><b> Mapped NAICS (North American Industry Classification System) codes to human-readable industry sectors and sub-sectors, enriching the dataset with more interpretable information which can be useful for analysis and modeling.</li></b><br>
<li><b>Identified and encoded risk groups based on NAICS codes, adding a Risk_Group feature to the dataset which categorizes businesses into different risk profiles.</li></b>

In [23]:
columns= ['franchise_name','naics_risk_group','loan_id', '2017_naics', 'naics_1', 'naics_2', 'naics_3', 'naics_4', 'naics_5']
nan_values = df[columns].isna().sum()
nan_values

franchise_name      95590
naics_risk_group    88313
loan_id                 0
2017_naics              0
naics_1                 0
naics_2                 0
naics_3                 0
naics_4                 0
naics_5                 0
dtype: int64

In [24]:
df['naics_risk_group'].value_counts()

Cash Intensive Businesses                 7717
High Risk                                 3321
NBFI (Non-Bank Financial Institutions)     622
MSB (Money Services Businesses)             27
Name: naics_risk_group, dtype: int64

In [25]:
df['naics_2'].unique()

array([72, 44, 33, 81, 23, 53, 54, 42, 52, 62, 32, 49, 48, 56, 71, 61, 45,
       31, 21, 51, 55, 11, 92, 22])

In [26]:
sector_mapping = {
    11: "Agriculture, Forestry, Fishing and Hunting",
    21: "Mining, Quarrying, and Oil and Gas Extraction",
    22: "Utilities",
    23: "Construction",
    31: "Manufacturing",
    32: "Manufacturing",
    33: "Manufacturing",
    42: "Wholesale Trade",
    44: "Retail Trade",
    45: "Retail Trade",
    48: "Transportation and Warehousing",
    49: "Transportation and Warehousing",
    51: "Information",
    52: "Finance and Insurance",
    53: "Real Estate and Rental and Leasing",
    54: "Professional, Scientific, and Technical Services",
    55: "Management of Companies and Enterprises",
    56: "Administrative and Support and Waste Management and Remediation Services",
    61: "Educational Services",
    62: "Health Care and Social Assistance",
    71: "Arts, Entertainment, and Recreation",
    72: "Accommodation and Food Services",
    81: "Other Services (except Public Administration)",
    92: "Public Administration"
}


df['Industry_Sector'] = df['naics_2'].map(sector_mapping)
df.head()

,loan_id,period_start,probability_of_default,loss_given_default,expected_loss,lumos_score,rate_type,franchise,franchise_name,borrstate,...,county_unemployment,state_unemployment,county_employment,state_employment,county_labor_force,state_labor_force,state_civilian_labor_population,state_employment_pop_ratio,state_labor_force_participation_rate,Industry_Sector
0,0,2020-07-31,1.38,18.39,0.25,100,F,False,NaN,KS,...,20696.0,105788.0,323149.0,1383213.0,343845.0,1489001.0,2247710.0,61.5,66.2,Accommodation and Food Services
1,1,2019-11-30,1.83,30.18,0.55,94,V,False,NaN,IL,...,13875.0,221403.0,491661.0,6341422.0,505536.0,6562825.0,10124996.0,62.6,64.8,Retail Trade
2,2,2015-08-31,5.05,66.34,3.35,81,V,False,NaN,CO,...,1130.0,104227.0,29960.0,2736370.0,31090.0,2840597.0,4237868.0,64.6,67.0,Retail Trade
3,3,2009-12-31,3.92,42.93,1.68,86,V,False,NaN,CT,...,42534.0,161898.0,424146.0,1730674.0,466680.0,1892572.0,2754731.0,62.8,68.7,Manufacturing
4,4,2018-10-31,2.96,47.49,1.41,88,F,False,NaN,CA,...,45641.0,759367.0,1573536.0,18567776.0,1619177.0,19327143.0,30994913.0,59.9,62.4,Accommodation and Food Services


In [27]:
pd.set_option('display.max_columns', None)
print(df.head(5))

   loan_id period_start  probability_of_default  loss_given_default  \
0        0   2020-07-31                    1.38               18.39   
1        1   2019-11-30                    1.83               30.18   
2        2   2015-08-31                    5.05               66.34   
3        3   2009-12-31                    3.92               42.93   
4        4   2018-10-31                    2.96               47.49   

   expected_loss  lumos_score rate_type  franchise franchise_name borrstate  \
0           0.25          100         F      False            NaN        KS   
1           0.55           94         V      False            NaN        IL   
2           3.35           81         V      False            NaN        CO   
3           1.68           86         V      False            NaN        CT   
4           1.41           88         F      False            NaN        CA   

  businesstype      business_type  empoyessexistingqty  \
0  CORPORATION  EXISTING BUSINESS       

In [28]:
df['Industry_Sector'].value_counts()

Retail Trade                                                                14572
Construction                                                                12401
Professional, Scientific, and Technical Services                            12303
Accommodation and Food Services                                              9711
Other Services (except Public Administration)                                9521
Health Care and Social Assistance                                            7681
Manufacturing                                                                6993
Administrative and Support and Waste Management and Remediation Services     5834
Wholesale Trade                                                              5429
Transportation and Warehousing                                               4927
Real Estate and Rental and Leasing                                           2623
Arts, Entertainment, and Recreation                                          2326
Finance and Insu

In [29]:
sub_sector_mapping = {
    111: "Crop Production",
    112: "Animal Production and Aquaculture",
    113: "Forestry and Logging",
    114: "Fishing, Hunting and Trapping",
    115: "Support Activities for Agriculture and Forestry",
    211: "Oil and Gas Extraction",
    212: "Mining (except Oil and Gas)",
    213: "Support Activities for Mining",
    221: "Utilities",
    236: "Construction of Buildings",
    237: "Heavy and Civil Engineering Construction",
    238: "Specialty Trade Contractors",
    311: "Food Manufacturing",
    312: "Beverage and Tobacco Product Manufacturing",
    313: "Textile Mills",
    314: "Textile Product Mills",
    315: "Apparel Manufacturing",
    316: "Leather and Allied Product Manufacturing",
    321: "Wood Product Manufacturing",
    322: "Paper Manufacturing",
    323: "Printing and Related Support Activities",
    324: "Petroleum and Coal Products Manufacturing",
    325: "Chemical Manufacturing",
    326: "Plastics and Rubber Products Manufacturing",
    327: "Nonmetallic Mineral Product Manufacturing",
    331: "Primary Metal Manufacturing",
    332: "Fabricated Metal Product Manufacturing",
    333: "Machinery Manufacturing",
    334: "Computer and Electronic Product Manufacturing",
    335: "Electrical Equipment, Appliance, and Component Manufacturing",
    336: "Transportation Equipment Manufacturing",
    337: "Furniture and Related Product Manufacturing",
    339: "Miscellaneous Manufacturing",
    423: "Merchant Wholesalers, Durable Goods",
    424: "Merchant Wholesalers, Nondurable Goods",
    425: "Wholesale Electronic Markets and Agents and Brokers",
    441: "Motor Vehicle and Parts Dealers",
    442: "Furniture and Home Furnishings Stores",
    443: "Electronics and Appliance Stores",
    444: "Building Material and Garden Equipment and Supplies Dealers",
    445: "Food and Beverage Stores",
    446: "Health and Personal Care Stores",
    447: "Gasoline Stations",
    448: "Clothing and Clothing Accessories Stores",
    451: "Sporting Goods, Hobby, Musical Instrument, and Book Stores",
    452: "General Merchandise Stores",
    453: "Miscellaneous Store Retailers",
    454: "Nonstore Retailers",
    481: "Air Transportation",
    482: "Rail Transportation",
    483: "Water Transportation",
    484: "Truck Transportation",
    485: "Transit and Ground Passenger Transportation",
    486: "Pipeline Transportation",
    487: "Scenic and Sightseeing Transportation",
    488: "Support Activities for Transportation",
    491: "Postal Service",
    492: "Couriers and Messengers",
    493: "Warehousing and Storage",
    511: "Publishing Industries (except Internet)",
    512: "Motion Picture and Sound Recording Industries",
    515: "Broadcasting (except Internet)",
    517: "Telecommunications",
    518: "Data Processing, Hosting, and Related Services",
    519: "Other Information Services",
    521: "Monetary Authorities-Central Bank",
    522: "Credit Intermediation and Related Activities",
    523: "Securities, Commodity Contracts, and Other Financial Investments and Related Activities",
    524: "Insurance Carriers and Related Activities",
    525: "Funds, Trusts, and Other Financial Vehicles",
    531: "Real Estate",
    532: "Rental and Leasing Services",
    533: "Lessors of Nonfinancial Intangible Assets",
    541: "Professional, Scientific, and Technical Services",
    551: "Management of Companies and Enterprises",
    561: "Administrative and Support Services",
    562: "Waste Management and Remediation Services",
    611: "Educational Services",
    621: "Ambulatory Health Care Services",
    622: "Hospitals",
    623: "Nursing and Residential Care Facilities",
    624: "Social Assistance",
    711: "Performing Arts, Spectator Sports, and Related Industries",
    712: "Museums, Historical Sites, and Similar Institutions",
    713: "Amusement, Gambling, and Recreation Industries",
    721: "Accommodation",
    722: "Food Services and Drinking Places",
    811: "Repair and Maintenance",
    812: "Personal and Laundry Services",
    813: "Religious, Grantmaking, Civic, Professional, and Similar Organizations",
    814: "Private Households",
    921: "Executive, Legislative, and Other General Government Support",
    922: "Justice, Public Order, and Safety Activities",
    923: "Administration of Human Resource Programs",
    924: "Administration of Environmental Quality Programs",
    925: "Administration of Housing Programs, Urban Planning, and Community Development",
    926: "Administration of Economic Programs",
    927: "Space Research and Technology",
    928: "National Security and International Affairs"
}


df['Industry_Sub_Sector'] = df['naics_3'].map(sub_sector_mapping)
df.head()

,loan_id,period_start,probability_of_default,loss_given_default,expected_loss,lumos_score,rate_type,franchise,franchise_name,borrstate,businesstype,business_type,empoyessexistingqty,naics_risk_group,2017_naics,naics_1,naics_2,naics_3,naics_4,naics_5,borr_count,loan_max_fico,loan_min_fico,loan_mean_fico,collateral,primary_collateral_type,collateral_amount,loan_size,business_revenue,bankruptcy,lawsuit,current_inflation_rate,expected_12mth_inflation_rate,fed_funds,t10yie,vix,vix_change,avg_hourly_earnings,t10y2y,county_unemployment_rate,state_unemployment_rate,county_unemployment,state_unemployment,county_employment,state_employment,county_labor_force,state_labor_force,state_civilian_labor_population,state_employment_pop_ratio,state_labor_force_participation_rate,Industry_Sector,Industry_Sub_Sector
0,0,2020-07-31,1.38,18.39,0.25,100,F,False,NaN,KS,CORPORATION,EXISTING BUSINESS,6,NaN,722511,7,72,722,7225,72251,2,791,727,759.0,True,commercial_re_1e,0-200k,101k-500k,432000,N,N,1.71,3.0,0.10,1.55,24.46,36.88,4.92,0.44,6.0,7.1,20696.0,105788.0,323149.0,1383213.0,343845.0,1489001.0,2247710.0,61.5,66.2,Accommodation and Food Services,Food Services and Drinking Places
1,1,2019-11-30,1.83,30.18,0.55,94,V,False,NaN,IL,CORPORATION,NEW BUSINESS,5,NaN,446120,4,44,446,4461,44612,2,770,759,764.5,True,commercial_re_1e,201k-1M,0-100k,0,N,N,2.70,2.5,1.56,1.61,12.62,-30.16,3.35,0.17,2.7,3.4,13875.0,221403.0,491661.0,6341422.0,505536.0,6562825.0,10124996.0,62.6,64.8,Retail Trade,Health and Personal Care Stores
2,2,2015-08-31,5.05,66.34,3.35,81,V,False,NaN,CO,CORPORATION,NEW BUSINESS,1,Cash Intensive Businesses,447110,4,44,447,4471,44711,1,689,689,689.0,True,other_4a,201k-1M,0-100k,0,N,N,1.92,2.8,0.08,1.63,28.43,137.31,2.20,1.47,3.6,3.7,1130.0,104227.0,29960.0,2736370.0,31090.0,2840597.0,4237868.0,64.6,67.0,Retail Trade,Gasoline Stations
3,3,2009-12-31,3.92,42.93,1.68,86,V,False,NaN,CT,CORPORATION,EXISTING BUSINESS,2,NaN,332322,3,33,332,3323,33232,2,732,701,716.5,True,commercial_re_1e,201k-1M,501k+,223000,N,N,0.52,2.7,0.05,2.37,21.68,-45.80,1.91,2.71,9.1,8.6,42534.0,161898.0,424146.0,1730674.0,466680.0,1892572.0,2754731.0,62.8,68.7,Manufacturing,Fabricated Metal Product Manufacturing
4,4,2018-10-31,2.96,47.49,1.41,88,F,False,NaN,CA,CORPORATION,NEW BUSINESS,1,NaN,721110,7,72,721,7211,72111,1,734,734,734.0,True,commercial_re_1e,201k-1M,501k+,0,N,N,2.68,2.7,2.20,2.05,21.23,108.55,3.28,0.28,2.8,3.9,45641.0,759367.0,1573536.0,18567776.0,1619177.0,19327143.0,30994913.0,59.9,62.4,Accommodation and Food Services,Accommodation


In [30]:
df['Industry_Sub_Sector'].value_counts()

Professional, Scientific, and Technical Services                12303
Food Services and Drinking Places                                9393
Specialty Trade Contractors                                      8381
Ambulatory Health Care Services                                  6292
Administrative and Support Services                              5397
                                                                ...  
Administration of Economic Programs                                 5
Administration of Human Resource Programs                           5
National Security and International Affairs                         4
Rail Transportation                                                 4
Executive, Legislative, and Other General Government Support        1
Name: Industry_Sub_Sector, Length: 96, dtype: int64

In [31]:
risk_group_mapping = {
    # Cash Intensive Businesses
    424940: 'Cash Intensive Businesses', 444110: 'Cash Intensive Businesses',
    444120: 'Cash Intensive Businesses', 444230: 'Cash Intensive Businesses',
    444240: 'Cash Intensive Businesses', 445131: 'Cash Intensive Businesses',
    445132: 'Cash Intensive Businesses', 445320: 'Cash Intensive Businesses',
    449110: 'Cash Intensive Businesses', 449121: 'Cash Intensive Businesses',
    449129: 'Cash Intensive Businesses', 449210: 'Cash Intensive Businesses',
    455110: 'Cash Intensive Businesses', 456110: 'Cash Intensive Businesses',
    457110: 'Cash Intensive Businesses', 458110: 'Cash Intensive Businesses',
    458210: 'Cash Intensive Businesses', 459110: 'Cash Intensive Businesses',
    459120: 'Cash Intensive Businesses', 459130: 'Cash Intensive Businesses',
    459140: 'Cash Intensive Businesses', 459210: 'Cash Intensive Businesses',
    459310: 'Cash Intensive Businesses', 459410: 'Cash Intensive Businesses',
    459420: 'Cash Intensive Businesses', 459510: 'Cash Intensive Businesses',
    459999: 'Cash Intensive Businesses', 722511: 'Cash Intensive Businesses',
    722513: 'Cash Intensive Businesses', 722514: 'Cash Intensive Businesses',
    812930: 'Cash Intensive Businesses', 813219: 'Cash Intensive Businesses',
    
    # HIGH RISK
    441110: 'High Risk', 441120: 'High Risk', 441210: 'High Risk',
    441222: 'High Risk', 441227: 'High Risk', 441330: 'High Risk',
    561510: 'High Risk', 713210: 'High Risk', 811111: 'High Risk',
    811114: 'High Risk', 811121: 'High Risk',
    
    # MSBs (Money Services Businesses)
    522291: 'MSB', 522320: 'MSB', 522390: 'MSB', 523160: 'MSB',
    
    # NBFIs (Non-Bank Financial Institutions)
    423940: 'NBFI', 458310: 'NBFI', 522299: 'NBFI',
}


df['Risk_Group'] = df['2017_naics'].map(risk_group_mapping).fillna('Standard Risk')
df.head()

,loan_id,period_start,probability_of_default,loss_given_default,expected_loss,lumos_score,rate_type,franchise,franchise_name,borrstate,businesstype,business_type,empoyessexistingqty,naics_risk_group,2017_naics,naics_1,naics_2,naics_3,naics_4,naics_5,borr_count,loan_max_fico,loan_min_fico,loan_mean_fico,collateral,primary_collateral_type,collateral_amount,loan_size,business_revenue,bankruptcy,lawsuit,current_inflation_rate,expected_12mth_inflation_rate,fed_funds,t10yie,vix,vix_change,avg_hourly_earnings,t10y2y,county_unemployment_rate,state_unemployment_rate,county_unemployment,state_unemployment,county_employment,state_employment,county_labor_force,state_labor_force,state_civilian_labor_population,state_employment_pop_ratio,state_labor_force_participation_rate,Industry_Sector,Industry_Sub_Sector,Risk_Group
0,0,2020-07-31,1.38,18.39,0.25,100,F,False,NaN,KS,CORPORATION,EXISTING BUSINESS,6,NaN,722511,7,72,722,7225,72251,2,791,727,759.0,True,commercial_re_1e,0-200k,101k-500k,432000,N,N,1.71,3.0,0.10,1.55,24.46,36.88,4.92,0.44,6.0,7.1,20696.0,105788.0,323149.0,1383213.0,343845.0,1489001.0,2247710.0,61.5,66.2,Accommodation and Food Services,Food Services and Drinking Places,Cash Intensive Businesses
1,1,2019-11-30,1.83,30.18,0.55,94,V,False,NaN,IL,CORPORATION,NEW BUSINESS,5,NaN,446120,4,44,446,4461,44612,2,770,759,764.5,True,commercial_re_1e,201k-1M,0-100k,0,N,N,2.70,2.5,1.56,1.61,12.62,-30.16,3.35,0.17,2.7,3.4,13875.0,221403.0,491661.0,6341422.0,505536.0,6562825.0,10124996.0,62.6,64.8,Retail Trade,Health and Personal Care Stores,Standard Risk
2,2,2015-08-31,5.05,66.34,3.35,81,V,False,NaN,CO,CORPORATION,NEW BUSINESS,1,Cash Intensive Businesses,447110,4,44,447,4471,44711,1,689,689,689.0,True,other_4a,201k-1M,0-100k,0,N,N,1.92,2.8,0.08,1.63,28.43,137.31,2.20,1.47,3.6,3.7,1130.0,104227.0,29960.0,2736370.0,31090.0,2840597.0,4237868.0,64.6,67.0,Retail Trade,Gasoline Stations,Standard Risk
3,3,2009-12-31,3.92,42.93,1.68,86,V,False,NaN,CT,CORPORATION,EXISTING BUSINESS,2,NaN,332322,3,33,332,3323,33232,2,732,701,716.5,True,commercial_re_1e,201k-1M,501k+,223000,N,N,0.52,2.7,0.05,2.37,21.68,-45.80,1.91,2.71,9.1,8.6,42534.0,161898.0,424146.0,1730674.0,466680.0,1892572.0,2754731.0,62.8,68.7,Manufacturing,Fabricated Metal Product Manufacturing,Standard Risk
4,4,2018-10-31,2.96,47.49,1.41,88,F,False,NaN,CA,CORPORATION,NEW BUSINESS,1,NaN,721110,7,72,721,7211,72111,1,734,734,734.0,True,commercial_re_1e,201k-1M,501k+,0,N,N,2.68,2.7,2.20,2.05,21.23,108.55,3.28,0.28,2.8,3.9,45641.0,759367.0,1573536.0,18567776.0,1619177.0,19327143.0,30994913.0,59.9,62.4,Accommodation and Food Services,Accommodation,Standard Risk


In [32]:
df['Risk_Group'].value_counts()

Standard Risk                89410
Cash Intensive Businesses     7650
High Risk                     2734
NBFI                           179
MSB                             27
Name: Risk_Group, dtype: int64

In [33]:
df.columns

Index(['loan_id', 'period_start', 'probability_of_default',
       'loss_given_default', 'expected_loss', 'lumos_score', 'rate_type',
       'franchise', 'franchise_name', 'borrstate', 'businesstype',
       'business_type', 'empoyessexistingqty', 'naics_risk_group',
       '2017_naics', 'naics_1', 'naics_2', 'naics_3', 'naics_4', 'naics_5',
       'borr_count', 'loan_max_fico', 'loan_min_fico', 'loan_mean_fico',
       'collateral', 'primary_collateral_type', 'collateral_amount',
       'loan_size', 'business_revenue', 'bankruptcy', 'lawsuit',
       'current_inflation_rate', 'expected_12mth_inflation_rate', 'fed_funds',
       't10yie', 'vix', 'vix_change', 'avg_hourly_earnings', 't10y2y',
       'county_unemployment_rate', 'state_unemployment_rate',
       'county_unemployment', 'state_unemployment', 'county_employment',
       'state_employment', 'county_labor_force', 'state_labor_force',
       'state_civilian_labor_population', 'state_employment_pop_ratio',
       'state_labor_fo

In [34]:
df.drop(columns=['naics_risk_group'], inplace=True)

Handling missing values for the columns

'county_unemployment_rate','state_unemployment','county_unemployment',
   'state_unemployment','county_employment','state_employment',
   'county_labor_force','state_labor_force','state_civilian_labor_population',
   'state_employment_pop_ratio','state_labor_force_participation_rate'

In [35]:
df[['county_unemployment_rate','state_unemployment','county_unemployment',
   'state_unemployment','county_employment','state_employment',
   'county_labor_force','state_labor_force','state_civilian_labor_population',
   'state_employment_pop_ratio','state_labor_force_participation_rate']].isnull().groupby(df['borrstate']).sum()

,county_unemployment_rate,state_unemployment,county_unemployment,state_unemployment,county_employment,state_employment,county_labor_force,state_labor_force,state_civilian_labor_population,state_employment_pop_ratio,state_labor_force_participation_rate
borrstate,,,,,,,,,,,
AK,2,0,2,0,2,0,2,0,0,0,0
AL,0,0,0,0,0,0,0,0,0,0,0
AR,0,0,0,0,0,0,0,0,0,0,0
AZ,0,0,0,0,0,0,0,0,0,0,0
CA,0,0,0,0,0,0,0,0,0,0,0
CO,0,0,0,0,0,0,0,0,0,0,0
CT,0,0,0,0,0,0,0,0,0,0,0
DC,0,0,0,0,0,0,0,0,0,0,0
DE,0,0,0,0,0,0,0,0,0,0,0


In [36]:
df[['county_unemployment_rate','state_unemployment','county_unemployment',
   'state_unemployment','county_employment','state_employment',
   'county_labor_force','state_labor_force','state_civilian_labor_population',
   'state_employment_pop_ratio','state_labor_force_participation_rate']].groupby(df['borrstate']).count()

,county_unemployment_rate,state_unemployment,county_unemployment,state_unemployment,county_employment,state_employment,county_labor_force,state_labor_force,state_civilian_labor_population,state_employment_pop_ratio,state_labor_force_participation_rate
borrstate,,,,,,,,,,,
AK,66,68,66,68,66,68,66,68,68,68,68
AL,338,338,338,338,338,338,338,338,338,338,338
AR,288,288,288,288,288,288,288,288,288,288,288
AZ,2192,2192,2192,2192,2192,2192,2192,2192,2192,2192,2192
CA,9116,9116,9116,9116,9116,9116,9116,9116,9116,9116,9116
CO,2364,2364,2364,2364,2364,2364,2364,2364,2364,2364,2364
CT,1682,1682,1682,1682,1682,1682,1682,1682,1682,1682,1682
DC,147,147,147,147,147,147,147,147,147,147,147
DE,345,345,345,345,345,345,345,345,345,345,345


In [37]:
df['borrstate'].unique()

array(['KS', 'IL', 'CO', 'CT', 'CA', 'AZ', 'NY', 'MD', 'FL', 'UT', 'MO',
       'IN', 'OR', 'NE', 'WV', 'NC', 'PA', 'WI', 'ID', 'MA', 'MI', 'VA',
       'NH', 'OK', 'TN', 'NJ', 'TX', 'KY', 'PR', 'MN', 'GA', 'ME', 'WA',
       'ND', 'OH', 'DE', 'RI', 'DC', 'VT', 'IA', 'AL', 'LA', 'AR', 'NV',
       'HI', 'WY', 'MT', 'SC', 'MS', 'NM', 'AK', 'SD', 'GU', 'VI'],
      dtype=object)

### Handling Missing Values¶

In [39]:
# List of columns in the dataset that require imputation for missing values.
columns_to_impute = [
    'county_unemployment_rate', 'county_unemployment',
    'state_unemployment', 'county_employment', 'state_employment',
    'county_labor_force', 'state_labor_force'
]

def impute_values_state(data, columns,state):
    """
    Imputes missing values for specified columns using median values from state).
    
    This function targets rows for the mentioned borrstate, calculating
    median values for each specified column within state and filling missing values with these medians.
    
    Parameters:
    - data (pd.DataFrame): The dataset containing the columns to be imputed.
    - columns (list): A list of column names where missing values are to be imputed.
    
    The function modifies the 'data' DataFrame in place; no return is necessary.
    """

    # Filter the DataFrame for the state
    df_state = data[data['borrstate'] == state]

    # Calculate the median values for each specified column for that state
    medians = df_state[columns].median()

    # Impute missing values in the original DataFrame for that state
    for column in columns:
        # Fill missing values with the calculated median for that column
        data.loc[(data['borrstate'] == state) & (data[column].isnull()), column] = medians[column]
        
    return data


# Specific estimated values to impute for missing data in PR, GU, and VI.
Imputing_specific_estimates = {
    'county_unemployment_rate': 2.5,
    'state_unemployment_rate': 3.0,
    'county_unemployment': 16764.0,
    'state_unemployment': 139700.0,
    'county_employment': 78870.0,
    'state_employment': 4451800.0,
    'county_labor_force': 343845.0,
    'state_labor_force': 1489001.0,
    'state_civilian_labor_population': 4591500.0,
    'state_employment_pop_ratio': 61.5,
    'state_labor_force_participation_rate': 66.6
}


def impute_missing_values(data, impute_values):
    """
    Imputes missing values with specific estimates for selected geographic locations.
    
    This function targets rows where the 'borrstate' is 'PR', 'GU', or 'VI',
    filling missing values with predefined estimates for certain columns.
    
    Parameters:
    - data (pd.DataFrame): The dataset containing the columns to be imputed.
    - impute_values (dict): A dictionary where keys are column names and values are the 
      specific values to use for imputation.
    
    Returns:
    - pd.DataFrame: The modified DataFrame with missing values imputed.
    """
    for column in impute_values.keys():
        # Fill missing values with the specified values for that column
        data.loc[((data['borrstate'] == 'PR') | (data['borrstate'] == 'GU') |
                  (data['borrstate'] == 'VI')) & (data[column].isnull()), column] = impute_values[column]

    return data


# Example usage of the defined functions to impute missing values in training, validation, and test sets.
df = impute_missing_values(df, Imputing_specific_estimates)

df= impute_values_state(df, ['county_unemployment_rate', 'county_unemployment', 'county_employment','county_labor_force'],'AK')

df= impute_values_state(df, ['county_unemployment_rate', 'county_unemployment', 'county_employment','county_labor_force'],'LA')

# Checking for remaining missing values in specific columns after imputation.
df[['county_unemployment_rate', 'state_unemployment', 'county_unemployment',
         'state_unemployment', 'county_employment', 'state_employment',
         'county_labor_force', 'state_labor_force', 'state_civilian_labor_population',
         'state_employment_pop_ratio', 'state_labor_force_participation_rate']].isnull().groupby(df['borrstate']).sum()


,county_unemployment_rate,state_unemployment,county_unemployment,state_unemployment,county_employment,state_employment,county_labor_force,state_labor_force,state_civilian_labor_population,state_employment_pop_ratio,state_labor_force_participation_rate
borrstate,,,,,,,,,,,
AK,0,0,0,0,0,0,0,0,0,0,0
AL,0,0,0,0,0,0,0,0,0,0,0
AR,0,0,0,0,0,0,0,0,0,0,0
AZ,0,0,0,0,0,0,0,0,0,0,0
CA,0,0,0,0,0,0,0,0,0,0,0
CO,0,0,0,0,0,0,0,0,0,0,0
CT,0,0,0,0,0,0,0,0,0,0,0
DC,0,0,0,0,0,0,0,0,0,0,0
DE,0,0,0,0,0,0,0,0,0,0,0


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 52 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   loan_id                               100000 non-null  int64         
 1   period_start                          100000 non-null  datetime64[ns]
 2   probability_of_default                100000 non-null  float64       
 3   loss_given_default                    100000 non-null  float64       
 4   expected_loss                         100000 non-null  float64       
 5   lumos_score                           100000 non-null  int64         
 6   rate_type                             100000 non-null  object        
 7   franchise                             100000 non-null  bool          
 8   franchise_name                        4410 non-null    object        
 9   borrstate                             100000 non-null  objec

#### Drop Columns

In [41]:
drop_columns= ['franchise_name','loan_id','period_start','borrstate','Industry_Sector','Industry_Sub_Sector']


df.drop(drop_columns, axis=1, inplace=True)


In [42]:
df[['collateral','primary_collateral_type','collateral_amount']]

,collateral,primary_collateral_type,collateral_amount
0,True,commercial_re_1e,0-200k
1,True,commercial_re_1e,201k-1M
2,True,other_4a,201k-1M
3,True,commercial_re_1e,201k-1M
4,True,commercial_re_1e,201k-1M
...,...,...,...
99995,False,NaN,NaN
99996,False,NaN,NaN
99997,False,NaN,NaN
99998,False,NaN,NaN


In [43]:
def impute_collateral(data):
    data['primary_collateral_type'] = data['primary_collateral_type'].fillna('Unknown')
    data['collateral_amount'] = data['collateral_amount'].fillna('0')
    
impute_collateral(df)


In [44]:
df[['collateral','primary_collateral_type','collateral_amount']]

,collateral,primary_collateral_type,collateral_amount
0,True,commercial_re_1e,0-200k
1,True,commercial_re_1e,201k-1M
2,True,other_4a,201k-1M
3,True,commercial_re_1e,201k-1M
4,True,commercial_re_1e,201k-1M
...,...,...,...
99995,False,Unknown,0
99996,False,Unknown,0
99997,False,Unknown,0
99998,False,Unknown,0


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 46 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   probability_of_default                100000 non-null  float64
 1   loss_given_default                    100000 non-null  float64
 2   expected_loss                         100000 non-null  float64
 3   lumos_score                           100000 non-null  int64  
 4   rate_type                             100000 non-null  object 
 5   franchise                             100000 non-null  bool   
 6   businesstype                          100000 non-null  object 
 7   business_type                         100000 non-null  object 
 8   empoyessexistingqty                   100000 non-null  int64  
 9   2017_naics                            100000 non-null  int64  
 10  naics_1                               100000 non-null  int64  
 11  n

In [46]:
missing_cols = ['avg_hourly_earnings']

In [47]:
df[missing_cols].describe()

,avg_hourly_earnings
count,94173.000000
mean,2.786268
std,0.908266
min,1.550000
25%,2.150000
50%,2.650000
75%,3.200000
max,8.030000


In [48]:
def missing_col(data):
    for col in missing_cols:
        median_value = data[col].median()
        data[col].fillna(median_value, inplace=True)

In [49]:
missing_col(df)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 46 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   probability_of_default                100000 non-null  float64
 1   loss_given_default                    100000 non-null  float64
 2   expected_loss                         100000 non-null  float64
 3   lumos_score                           100000 non-null  int64  
 4   rate_type                             100000 non-null  object 
 5   franchise                             100000 non-null  bool   
 6   businesstype                          100000 non-null  object 
 7   business_type                         100000 non-null  object 
 8   empoyessexistingqty                   100000 non-null  int64  
 9   2017_naics                            100000 non-null  int64  
 10  naics_1                               100000 non-null  int64  
 11  n

### Ordinal Categories


The ordinal_encoding function is designed to encode categorical variables into ordinal numerical formats based on predefined mappings.

In [51]:
df['collateral_amount'].value_counts()

0          39882
0-200k     20103
201k-1M    20065
1M+        19950
Name: collateral_amount, dtype: int64

In [52]:
df['franchise'] = df['franchise'].astype(int)
df['collateral'] = df['collateral'].astype(int)

In [53]:
def ordinal_encoding(data):
    loan_size_mapping = {
        '0-100k': 1,
        '101k-500k': 2,
        '501k+': 3
    }
    data['loan_size'] = data['loan_size'].map(loan_size_mapping)


    collateral_amount_mapping = {
        '0':1,
        '0-200k':2,
        '201k-1M':3,
        '1M+':4

    }
    data['collateral_amount'] = data['collateral_amount'].map(collateral_amount_mapping)
    
    
    
    return data

In [54]:
df = ordinal_encoding(df)

## Factorization

In [56]:
df['Risk_Group'] = pd.Categorical(df['Risk_Group']).codes
df['lawsuit'] = pd.Categorical(df['lawsuit']).codes
df['bankruptcy'] = pd.Categorical(df['bankruptcy']).codes
df['primary_collateral_type'] = pd.Categorical(df['primary_collateral_type']).codes
df['business_type'] = pd.Categorical(df['business_type']).codes
df['businesstype'] = pd.Categorical(df['businesstype']).codes
df['rate_type'] = pd.Categorical(df['rate_type']).codes

### Dropping naics code attributes

In [57]:
df=df.drop(['2017_naics', 'naics_1','naics_2', 'naics_3', 'naics_4', 'naics_5'], axis = 1)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 40 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   probability_of_default                100000 non-null  float64
 1   loss_given_default                    100000 non-null  float64
 2   expected_loss                         100000 non-null  float64
 3   lumos_score                           100000 non-null  int64  
 4   rate_type                             100000 non-null  int8   
 5   franchise                             100000 non-null  int64  
 6   businesstype                          100000 non-null  int8   
 7   business_type                         100000 non-null  int8   
 8   empoyessexistingqty                   100000 non-null  int64  
 9   borr_count                            100000 non-null  int64  
 10  loan_max_fico                         100000 non-null  int64  
 11  l

## concatenate the two DataFrames along the columns

In [59]:
df2 = pd.concat([df, df_exp], axis=1)
print(df2.head())

   probability_of_default  loss_given_default  expected_loss  lumos_score  \
0                    1.38               18.39           0.25          100   
1                    1.83               30.18           0.55           94   
2                    5.05               66.34           3.35           81   
3                    3.92               42.93           1.68           86   
4                    2.96               47.49           1.41           88   

   rate_type  franchise  businesstype  business_type  empoyessexistingqty  \
0          0          0             0              0                    6   
1          1          0             0              1                    5   
2          1          0             0              1                    1   
3          1          0             0              0                    2   
4          0          0             0              1                    1   

   borr_count  loan_max_fico  loan_min_fico  loan_mean_fico  collateral  \

In [60]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 50 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   probability_of_default                100000 non-null  float64
 1   loss_given_default                    100000 non-null  float64
 2   expected_loss                         100000 non-null  float64
 3   lumos_score                           100000 non-null  int64  
 4   rate_type                             100000 non-null  int8   
 5   franchise                             100000 non-null  int64  
 6   businesstype                          100000 non-null  int8   
 7   business_type                         100000 non-null  int8   
 8   empoyessexistingqty                   100000 non-null  int64  
 9   borr_count                            100000 non-null  int64  
 10  loan_max_fico                         100000 non-null  int64  
 11  l

In [61]:
df2.to_csv('cleaned_df.csv')

In [68]:

# Create an empty DataFrame to hold the renamed columns
df_exp1 = pd.DataFrame()

# Loop through the explanation indices (1 through 10, assuming they all exist and follow this format)
for i in range(1, 6):
    feature_col = f'EXPLANATION_{i}_FEATURE_NAME'
    
    # Check if the columns exist in df1, to handle cases where some indices may not be present
    if feature_col in df1.columns:
        # Copy the feature name column into the new DataFrame with a new column name
        df_exp1[f'EXPLANATION_{i}'] = df1[feature_col]

print(df_exp1)


        EXPLANATION_1                    EXPLANATION_2  EXPLANATION_3  \
0       business_type                        borrstate         t10y2y   
1      loan_mean_fico                    business_type  loan_max_fico   
2       loan_max_fico                   loan_mean_fico  business_type   
3              t10y2y                    loan_min_fico     bankruptcy   
4       loan_max_fico  state_civilian_labor_population  loan_min_fico   
...               ...                              ...            ...   
99995  loan_mean_fico                    loan_max_fico  loan_min_fico   
99996   loan_max_fico                          naics_5         t10y2y   
99997   loan_max_fico                   loan_mean_fico      borrstate   
99998          t10y2y                    business_type      borrstate   
99999      borr_count                          naics_2        naics_3   

                EXPLANATION_4             EXPLANATION_5  
0               loan_min_fico             loan_max_fico  
1      

In [69]:
df3= pd.concat([df, df_exp1], axis=1)


In [72]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 45 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   probability_of_default                100000 non-null  float64
 1   loss_given_default                    100000 non-null  float64
 2   expected_loss                         100000 non-null  float64
 3   lumos_score                           100000 non-null  int64  
 4   rate_type                             100000 non-null  int8   
 5   franchise                             100000 non-null  int64  
 6   businesstype                          100000 non-null  int8   
 7   business_type                         100000 non-null  int8   
 8   empoyessexistingqty                   100000 non-null  int64  
 9   borr_count                            100000 non-null  int64  
 10  loan_max_fico                         100000 non-null  int64  
 11  l

In [71]:
df3.to_csv('cleaned_df_recom.csv')

In [ ]:
EXPLANATION_1_QUALITATIVE_STRENGTH

In [77]:

# Create an empty DataFrame to hold the renamed columns
df_exp2 = pd.DataFrame()

# Loop through the explanation indices (1 through 10, assuming they all exist and follow this format)
for i in range(1, 6):
    feature_col = f'EXPLANATION_{i}_FEATURE_NAME'
    strength_col = f'EXPLANATION_{i}_STRENGTH'
    
    # Check if the columns exist in df1, to handle cases where some indices may not be present
    if feature_col in data.columns:
        # Copy the feature name column into the new DataFrame with a new column name
        df_exp2[f'EXPLANATION_{i}'] = data[feature_col]
        df_exp2[f'EXPLANATION_{i}_STRENGTH'] = data[strength_col]

print(df_exp2)

        EXPLANATION_1  EXPLANATION_1_STRENGTH  \
0       business_type               -0.165458   
1      loan_mean_fico               -0.255019   
2       loan_max_fico                0.264952   
3              t10y2y                0.180264   
4       loan_max_fico                0.311784   
...               ...                     ...   
99995  loan_mean_fico               -0.387256   
99996   loan_max_fico                0.158760   
99997   loan_max_fico                0.262102   
99998          t10y2y               -0.201739   
99999      borr_count               -0.479289   

                         EXPLANATION_2  EXPLANATION_2_STRENGTH  EXPLANATION_3  \
0                            borrstate                0.129951         t10y2y   
1                        business_type                0.239443  loan_max_fico   
2                       loan_mean_fico                0.202938  business_type   
3                        loan_min_fico               -0.119959     bankruptcy   
4     

In [78]:
df4 = pd.concat([df, df_exp2], axis=1)


In [79]:
df4.to_csv('cleaned_df_strength.csv')